In [49]:
from surprise import accuracy, Dataset, SVD, Reader, KNNBasic, AlgoBase

from surprise.model_selection import train_test_split, cross_validate, GridSearchCV

import pandas as pd

from collections import defaultdict

import numpy as np

# Завдання 1

In [3]:
# 1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise.

df = Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [10]:
# 2. Виведіть перші 5 рядків завантаженого датасету.

print('(№ користувача, № фільму, бал, часова позначка)')
for i in df.raw_ratings[:5]:
  print(i)

(№ користувача, № фільму, бал, часова позначка)
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [11]:
#train_df, test_df = train_test_split(df, test_size = 0.2)

## Реалізаіця двох алгоритмім (SVD та KNNBasic)

In [24]:
# Список для збереження результатів
benchmark = []

for algorithm in [SVD(), KNNBasic()]:
    # Крос-валідація
    results = cross_validate(algorithm, df, measures = ['RMSE', 'MAE'], cv = 3, verbose = False)

    # Отримання результаів та додавання імені алгоритму
    tmp = pd.DataFrame.from_dict(results).mean(axis = 0)

    # Об'єднання з tmp
    algorithm_name = pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index = ['Algorithm'])
    tmp = pd.concat([tmp, algorithm_name])

    # Додавання результатів до списку
    benchmark.append(tmp)

results_df = pd.DataFrame(benchmark).set_index('Algorithm')

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [25]:
results_df

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SVD,0.946216,0.746747,1.806213,0.436559
KNNBasic,0.989556,0.781758,0.386675,6.118700


In [26]:
# Сортування за RMSE для вибору найкращого алгоритму
best_algorithm_name = results_df.sort_values('test_rmse').index[0]
print(f"Найкращий алгоритм: {best_algorithm_name}")

Найкращий алгоритм: SVD


In [27]:
# Тренуємо найкращий алгоритм на всіх даних
if best_algorithm_name == 'SVD':
    best_algorithm = SVD()
elif best_algorithm_name == 'KNNBasic':
    best_algorithm = KNNBasic()

# Поділ на тренувальні і тестові дані
trainset, testset = train_test_split(df, test_size = 0.2)

# Тренування моделі
best_algorithm.fit(trainset)

# Оцінка точності на тестових даних
predictions = best_algorithm.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9378


0.9378215773165528

In [33]:
# Генерація рекомендацій для користувача
user_id      = '196'  # наприклад, для користувача з ID 196
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість оцінок користувача {user_id}: {len(user_ratings)}")

Кількість оцінок користувача 196: 31


In [35]:
# Отримання рекомендацій
all_items     = set(trainset.all_items())
rated_items   = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, best_algorithm.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key = lambda x: x[1], reverse = True)

# Виведення 10 найкращих фільмів
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} - Оцінка: {rating:.2f}")

Топ-10 фільмів, рекомендованих для користувача:
Фільм 318 - Оцінка: 4.59
Фільм 64 - Оцінка: 4.50
Фільм 197 - Оцінка: 4.49
Фільм 483 - Оцінка: 4.48
Фільм 357 - Оцінка: 4.46
Фільм 272 - Оцінка: 4.45
Фільм 513 - Оцінка: 4.37
Фільм 963 - Оцінка: 4.37
Фільм 474 - Оцінка: 4.36
Фільм 178 - Оцінка: 4.36


## Підбір гіперпараметрів

In [37]:
param_grid_svd = {
    'n_factors': [20, 50, 100],
    'n_epochs': [10, 20],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.05]
}

param_grid_knn = {
    'k': [20, 30, 40],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson_baseline'],
        'user_based': [False]
    }
}

# GridSearch для SVD
gs_svd = GridSearchCV(SVD, param_grid_svd, measures = ['rmse', 'mae'], cv = 3)
gs_svd.fit(df)

# GridSearch для KNNBasic
gs_knn = GridSearchCV(KNNBasic, param_grid_knn, measures = ['rmse', 'mae'], cv = 3)
gs_knn.fit(df)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity mat

In [45]:
# Порівняння результатів
best_svd_rmse = gs_svd.best_score['rmse']
best_knn_rmse = gs_knn.best_score['rmse']

print(f"Найкращий RMSE для SVD: {best_svd_rmse}")
print(f"Найкращий RMSE для KNNBasic: {best_knn_rmse}")

if best_svd_rmse < best_knn_rmse:
    best_algorithm_grid_s = SVD()
    best_algorithm_grid_s_name = 'SVD'
    best_params = gs_svd.best_params['rmse']
else:
    best_algorithm_grid_s = KNNBasic()
    best_algorithm_grid_s_name = 'KNNBasic'
    best_params = gs_knn.best_params['rmse']

print(f"Найкращий алгоритм: {best_algorithm_grid_s_name}")
print(f"Найкращі параметри: {best_params}")

Найкращий RMSE для SVD: 0.9318044344427531
Найкращий RMSE для KNNBasic: 1.0065864574137044
Найкращий алгоритм: SVD
Найкращі параметри: {'n_factors': 20, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.05}


In [46]:
# Тренування моделі
best_algorithm_grid_s.fit(trainset)

# Оцінка точності на тестових даних
predictions_grid_s = best_algorithm_grid_s.test(testset)
accuracy.rmse(predictions_grid_s)

RMSE: 0.9372


0.9372346113467963

In [47]:
# Отримання рекомендацій
all_items     = set(trainset.all_items())
rated_items   = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, best_algorithm_grid_s.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key = lambda x: x[1], reverse = True)

# Виведення 10 найкращих фільмів
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} - Оцінка: {rating:.2f}")

Топ-10 фільмів, рекомендованих для користувача:
Фільм 318 - Оцінка: 4.57
Фільм 496 - Оцінка: 4.43
Фільм 272 - Оцінка: 4.43
Фільм 603 - Оцінка: 4.40
Фільм 114 - Оцінка: 4.35
Фільм 427 - Оцінка: 4.34
Фільм 357 - Оцінка: 4.33
Фільм 169 - Оцінка: 4.33
Фільм 408 - Оцінка: 4.33
Фільм 178 - Оцінка: 4.31


# Завдання 2

## Побудова власної рекомендаційної системи

In [50]:
# Створення власного алгоритму
class MyOwnAlgorithm(AlgoBase):

    def __init__(self):
        # Викликаємо конструктор базового класу
        AlgoBase.__init__(self)

    def fit(self, trainset):
        # Викликаємо базовий метод fit()
        AlgoBase.fit(self, trainset)

        # Обчислюємо середній рейтинг для кожного користувача
        self.user_means = defaultdict(float)

        for uid, iid, rating in self.trainset.all_ratings():
            self.user_means[uid] += rating

        for uid in self.user_means:
            self.user_means[uid] /= self.trainset.n_items  # середній рейтинг користувача

        # Обчислюємо середній рейтинг для кожного предмета (item)
        self.item_means = defaultdict(float)

        for uid, iid, rating in self.trainset.all_ratings():
            self.item_means[iid] += rating

        for iid in self.item_means:
            self.item_means[iid] /= self.trainset.n_users  # середній рейтинг предмета

        return self

    def estimate(self, uid, iid):
        if self.trainset.knows_user(uid) and self.trainset.knows_item(iid):
            # Якщо знаємо і користувача, і предмет, беремо середнє значення між їхніми середніми рейтингами
            return (self.user_means[uid] + self.item_means[iid]) / 2
        else:
            # Якщо не знаємо користувача або предмет, повертаємо глобальне середнє
            return self.trainset.global_mean

In [51]:
# Завантажуємо вбудований набір даних
df = Dataset.load_builtin("ml-100k")

# Розбиваємо на тренувальні та тестові набори
trainset, testset = train_test_split(df, test_size = 0.2)

# Створюємо екземпляр нового алгоритму
my_algorithm = MyOwnAlgorithm()

# Тренуємо модель
my_algorithm.fit(trainset)

# Тестуємо на тестових даних
predictions = my_algorithm.test(testset)

# Оцінюємо якість моделі
accuracy.rmse(predictions)

RMSE: 2.7649


2.7648750510709763

In [52]:
# Можемо також отримати рекомендації для користувача
user_id = '196'  # Наприклад, користувач з ID 196
movie_id = '302'  # Наприклад, фільм з ID 302

# Прогнозуємо оцінку для даного фільму
prediction = my_algorithm.predict(user_id, movie_id)
print(f"Прогнозована оцінка для користувача {user_id} і фільму {movie_id}: {prediction.est:.2f}")

Прогнозована оцінка для користувача 196 і фільму 302: 1.00


In [53]:
# Отримання рекомендацій
all_items     = set(trainset.all_items())
rated_items   = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, my_algorithm.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key = lambda x: x[1], reverse = True)

# Виведення 10 найкращих фільмів
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} - Оцінка: {rating:.2f}")

Топ-10 фільмів, рекомендованих для користувача:
Фільм 50 - Оцінка: 1.12
Фільм 538 - Оцінка: 1.00
Фільм 180 - Оцінка: 1.00
Фільм 732 - Оцінка: 1.00
Фільм 476 - Оцінка: 1.00
Фільм 169 - Оцінка: 1.00
Фільм 411 - Оцінка: 1.00
Фільм 25 - Оцінка: 1.00
Фільм 234 - Оцінка: 1.00
Фільм 809 - Оцінка: 1.00
